# IT Service Desk and Support Operations Analytics
## Notebook 01: Data Profiling, KPI Foundations, and QA Checks

This notebook profiles the IT Service Management (ITSM) dataset and prepares it for analytics and reporting.

### Goals
- Confirm dataset structure and allowed values
- Parse time columns into usable datetime types
- Create derived KPI-ready columns (durations and flags)
- Run data quality checks (duplicates, chronology, SLA reconciliation)
- Export an analytics-ready dataset for SQL loading and BI dashboards

### Inputs and Outputs
- Input: `../data/raw/itsm_raw.csv`
- Output: `../data/processed/itsm_clean.csv`

### Notes
- This dataset is synthetic (educational). Results are still useful to demonstrate an end-to-end operational analytics pipeline.
- SLA targets are modeled as deadline timestamps (not durations). We compute SLA compliance by comparing actual timestamps to SLA deadlines.


In [1]:
# If you see a ModuleNotFoundError, activate your virtual environment and install requirements.
# In PowerShell (Windows):
#   Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass
#   .\.venv\Scripts\Activate.ps1
#   pip install -r requirements.txt

import pandas as pd
import numpy as np


## 1. Load the dataset

We load the raw CSV. The raw file is treated as read-only source data.


In [2]:
# Update this path only if your repo structure is different.
RAW_PATH = "../data/raw/itsm_raw.csv"

df = pd.read_csv(RAW_PATH)
df.shape


(100000, 22)

## 2. Basic inspection

We inspect column names, data types, and a sample of records. This helps confirm the dataset matches expectations.


In [3]:
# Column names
list(df.columns)


['Status',
 'Ticket ID',
 'Priority',
 'Source',
 'Topic',
 'Agent Group',
 'Agent Name',
 'Created time',
 'Expected SLA to resolve',
 'Expected SLA to first response',
 'First response time',
 'SLA For first response',
 'Resolution time',
 'SLA For Resolution',
 'Close time',
 'Agent interactions',
 'Survey results',
 'Product group',
 'Support Level',
 'Country',
 'Latitude',
 'Longitude']

In [4]:
# Data types
df.dtypes


Status                                str
Ticket ID                             str
Priority                              str
Source                                str
Topic                                 str
Agent Group                           str
Agent Name                            str
Created time                          str
Expected SLA to resolve               str
Expected SLA to first response        str
First response time                   str
SLA For first response                str
Resolution time                       str
SLA For Resolution                    str
Close time                            str
Agent interactions                  int64
Survey results                        str
Product group                         str
Support Level                         str
Country                               str
Latitude                          float64
Longitude                         float64
dtype: object

In [5]:
# Preview a few rows (subset of columns for readability)
df.head()


,Status,Ticket ID,Priority,Source,Topic,Agent Group,Agent Name,Created time,Expected SLA to resolve,Expected SLA to first response,...,Resolution time,SLA For Resolution,Close time,Agent interactions,Survey results,Product group,Support Level,Country,Latitude,Longitude
0,Closed,TCKT-100000,High,Email,General Inquiry,Security,Khalid Al-Salem,2024-07-04 12:42:00,2024-07-04 14:42:00,2024-07-04 13:12:00,...,2024-07-04 14:30:00,Met,2024-07-04 14:32:00,5,Neutral,Cloud,L3,Oman,25.1856,50.9447
1,Closed,TCKT-100001,High,Chat,Network Issue,Customer Service,Ahmed Al-Sabah,2024-05-23 20:03:00,2024-05-23 22:03:00,2024-05-23 20:33:00,...,2024-05-23 22:00:00,Met,2024-05-23 22:05:00,4,Dissatisfied,Cloud,L2,Qatar,23.2741,55.3867
2,In Progress,TCKT-100002,Low,Phone,General Inquiry,Development,Mohammed Al-Mansoori,2024-04-13 20:51:00,2024-04-14 00:51:00,2024-04-13 21:51:00,...,2024-04-14 00:47:00,Met,2024-04-14 00:51:00,3,Dissatisfied,Software,L1,Bahrain,23.6264,50.1302
3,Resolved,TCKT-100003,Critical,Chat,Access Request,Development,Mohammed Al-Khalifa,2024-05-13 12:50:00,2024-05-13 13:50:00,2024-05-13 13:00:00,...,2024-05-13 13:48:00,Met,2024-05-13 13:53:00,5,Dissatisfied,Network,L2,Kuwait,25.0736,54.8437
4,Closed,TCKT-100004,Critical,Portal,Hardware Failure,Customer Service,Hassan Al-Nasser,2024-06-19 22:51:00,2024-06-19 23:51:00,2024-06-19 23:01:00,...,2024-06-19 23:49:00,Met,2024-06-19 23:54:00,4,Neutral,Hardware,L3,Qatar,24.7362,51.4839


## 3. Missing values and uniqueness checks

Even if the dataset is clean, we verify:
- No unexpected nulls
- Ticket ID can be used as a primary key (no duplicates)


In [6]:
# Missing values per column
df.isna().sum().sort_values(ascending=False)


Status                            0
Ticket ID                         0
Priority                          0
Source                            0
Topic                             0
Agent Group                       0
Agent Name                        0
Created time                      0
Expected SLA to resolve           0
Expected SLA to first response    0
First response time               0
SLA For first response            0
Resolution time                   0
SLA For Resolution                0
Close time                        0
Agent interactions                0
Survey results                    0
Product group                     0
Support Level                     0
Country                           0
Latitude                          0
Longitude                         0
dtype: int64

In [7]:
# Duplicate check on the primary key candidate
dup_ticket_ids = df.duplicated(subset=["Ticket ID"]).sum()
print("Duplicate Ticket IDs:", dup_ticket_ids)


Duplicate Ticket IDs: 0


## 4. Allowed values and distributions

These checks confirm the observed allowed values for categorical fields.
They also help set expectations for dashboard filters and KPI groupings.


In [8]:
# Status distribution
df["Status"].value_counts()


Status
Resolved       20134
In Progress    20123
Closed         20015
New            20014
Open           19714
Name: count, dtype: int64

In [9]:
# Priority distribution
df["Priority"].value_counts()


Priority
Medium      25117
Critical    25045
Low         25014
High        24824
Name: count, dtype: int64

In [10]:
# Agent Group distribution (top 10)
df["Agent Group"].value_counts().head(10)


Agent Group
Development         20158
Network Ops         20144
Security            19985
Customer Service    19884
IT Support          19829
Name: count, dtype: int64

In [11]:
# Survey results distribution
df["Survey results"].value_counts()


Survey results
Dissatisfied    33409
Neutral         33345
Satisfied       33246
Name: count, dtype: int64

## 5. Parse time columns into datetime

Time columns arrive as strings in the raw CSV. We convert them to datetime so we can:
- Calculate durations (first response time, resolution time)
- Compare actual timestamps against SLA deadline timestamps
- Validate chronological ordering


In [12]:
time_cols = [
    "Created time",
    "First response time",
    "Resolution time",
    "Close time",
    "Expected SLA to first response",
    "Expected SLA to resolve",
]

# Convert to datetime; errors='raise' will surface formatting issues early
for c in time_cols:
    df[c] = pd.to_datetime(df[c], errors="raise")

# Confirm parsing produced no missing timestamps
df[time_cols].isna().sum()


Created time                      0
First response time               0
Resolution time                   0
Close time                        0
Expected SLA to first response    0
Expected SLA to resolve           0
dtype: int64

## 6. Create KPI-ready derived columns

Create reusable columns that will be used consistently across:
- Python analysis
- SQL queries
- Power BI or Excel dashboard measures

Derived columns:
- `first_response_minutes`
- `resolution_minutes`
- `response_sla_met_recalc`
- `resolution_sla_met_recalc`
- `backlog_flag`
- `satisfaction_score`


In [13]:
# Duration metrics in minutes
df["first_response_minutes"] = (df["First response time"] - df["Created time"]).dt.total_seconds() / 60
df["resolution_minutes"] = (df["Resolution time"] - df["Created time"]).dt.total_seconds() / 60

# SLA compliance (recalculated from timestamps and SLA deadlines)
df["response_sla_met_recalc"] = df["First response time"] <= df["Expected SLA to first response"]
df["resolution_sla_met_recalc"] = df["Resolution time"] <= df["Expected SLA to resolve"]

# Backlog definition (open work)
backlog_statuses = ["New", "Open", "In Progress"]
df["backlog_flag"] = df["Status"].isin(backlog_statuses)

# Satisfaction score for numeric analysis
satisfaction_map = {"Satisfied": 3, "Neutral": 2, "Dissatisfied": 1}
df["satisfaction_score"] = df["Survey results"].map(satisfaction_map)

df[["first_response_minutes", "resolution_minutes", "backlog_flag", "satisfaction_score"]].head()


,first_response_minutes,resolution_minutes,backlog_flag,satisfaction_score
0,20.0,108.0,False,2
1,22.0,117.0,False,1
2,50.0,236.0,True,1
3,0.0,58.0,False,1
4,9.0,58.0,False,2


## 7. QA checks

These checks make the pipeline trustworthy.

### 7.1 Duration integrity
Durations should never be negative.


In [14]:
print("Negative first_response_minutes:", (df["first_response_minutes"] < 0).sum())
print("Negative resolution_minutes:", (df["resolution_minutes"] < 0).sum())

df[["first_response_minutes", "resolution_minutes"]].describe()


Negative first_response_minutes: 0
Negative resolution_minutes: 0


,first_response_minutes,resolution_minutes
count,100000.000000,100000.000000
mean,31.256900,140.073900
std,18.765417,67.378202
min,0.000000,40.000000
25%,10.000000,60.000000
50%,35.000000,160.000000
75%,50.000000,220.000000
max,60.000000,240.000000


### 7.2 Chronological integrity

Expected ordering:
- Created time <= First response time <= Resolution time <= Close time

Any violations indicate inconsistent timestamps.


In [15]:
bad_created_to_response = (df["First response time"] < df["Created time"]).sum()
bad_response_to_resolution = (df["Resolution time"] < df["First response time"]).sum()
bad_resolution_to_close = (df["Close time"] < df["Resolution time"]).sum()

print("First response before Created time:", bad_created_to_response)
print("Resolution before First response:", bad_response_to_resolution)
print("Close time before Resolution time:", bad_resolution_to_close)


First response before Created time: 0
Resolution before First response: 0
Close time before Resolution time: 0


### 7.3 SLA reconciliation

The dataset contains provided SLA labels:
- `SLA For first response`
- `SLA For Resolution`

Validate them by converting labels to boolean flags and comparing against our recalculated SLA results.
This is a common enterprise QA practice to confirm KPI logic matches the source system.


In [16]:
# Convert provided SLA labels into boolean flags
df["response_sla_flag_bool"] = df["SLA For first response"].astype(str).str.strip().eq("Met")
df["resolution_sla_flag_bool"] = df["SLA For Resolution"].astype(str).str.strip().eq("Met")

print("Response SLA flag matches recalculation:")
print((df["response_sla_met_recalc"] == df["response_sla_flag_bool"]).value_counts())

print("\nResolution SLA flag matches recalculation:")
print((df["resolution_sla_met_recalc"] == df["resolution_sla_flag_bool"]).value_counts())


Response SLA flag matches recalculation:
True    100000
Name: count, dtype: int64

Resolution SLA flag matches recalculation:
True    100000
Name: count, dtype: int64


## 8. Quick KPI snapshots (sanity checks)

These are lightweight checks to confirm the derived columns behave as expected.
They are not final dashboard visuals yet.


In [17]:
# Median times are generally more robust than means when outliers exist
kpi_snapshot = {
    "tickets_total": len(df),
    "first_response_median_min": float(df["first_response_minutes"].median()),
    "resolution_median_min": float(df["resolution_minutes"].median()),
    "backlog_open_tickets": int(df["backlog_flag"].sum()),
    "satisfaction_avg_score": float(df["satisfaction_score"].mean()),
}
kpi_snapshot


{'tickets_total': 100000,
 'first_response_median_min': 35.0,
 'resolution_median_min': 160.0,
 'backlog_open_tickets': 59851,
 'satisfaction_avg_score': 1.99837}

In [18]:
# SLA compliance rates (recalculated)
response_sla_rate = df["response_sla_met_recalc"].mean()
resolution_sla_rate = df["resolution_sla_met_recalc"].mean()

print("Response SLA compliance rate:", response_sla_rate)
print("Resolution SLA compliance rate:", resolution_sla_rate)


Response SLA compliance rate: 1.0
Resolution SLA compliance rate: 1.0


## 9. Export analytics-ready dataset

Export a processed dataset that includes derived columns and validated timestamps.
This file is used for:
- Loading into PostgreSQL (fact table)
- Power BI or Excel dashboards
- Reproducible pipeline execution

Raw data remains unchanged in `data/raw`.


In [19]:
PROCESSED_PATH = "../data/processed/itsm_clean.csv"

# Ensure output folder exists
from pathlib import Path
Path("../data/processed").mkdir(parents=True, exist_ok=True)

df.to_csv(PROCESSED_PATH, index=False)
print("Saved:", PROCESSED_PATH)


Saved: ../data/processed/itsm_clean.csv
